In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
import os
import datetime
from sklearn.preprocessing import StandardScaler
import re
from tensorflow.keras.layers import Dropout

%load_ext tensorboard

<h2> Preparing Training Set</h2>

In [2]:
def data_preprocessing_train(dataframe):
    # Drop the following columns
    df_dropped = dataframe.drop(['PassengerId', 'Ticket'], axis=1)
        
    # Fix Names
    df_dropped['Title'] = df_dropped.Name.str.extract(' ([A-Za-z]+)\.', expand = False)
    Title_Dictionary = {
            "Capt":       "Officer",
            "Col":        "Officer",
            "Major":      "Officer",
            "Dr":         "Officer",
            "Rev":        "Officer",
            "Jonkheer":   "Royalty",
            "Don":        "Royalty",
            "Sir" :       "Royalty",
            "the Countess":"Royalty",
            "Dona":       "Royalty",
            "Lady" :      "Royalty",
            "Mme":        "Mrs",
            "Ms":         "Mrs",
            "Mrs" :       "Mrs",
            "Mlle":       "Miss",
            "Miss" :      "Miss",
            "Mr" :        "Mr",
            "Master" :    "Master"
                       }
    
    df_dropped['Title'] = df_dropped.Title.map(Title_Dictionary)


    # Fix embarked
    df_dropped["Embarked"] = df_dropped["Embarked"].fillna(value="C")
    
    # Fix cabin
    df_dropped['Cabin'] = df_dropped['Cabin'].fillna('U')
    df_dropped['Cabin'] = df_dropped['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())



    column = ['Pclass','Sex','Embarked','SibSp','Parch','Survived']
    
    for value in range(len(column), 0, -1):
        df_dropped['Age'] = df_dropped['Age'].fillna(df_dropped.groupby(column[0:value])['Age'].transform('mean'))
        df_dropped['Fare'] = df_dropped['Fare'].fillna(df_dropped.groupby(column[0:value])['Fare'].transform('mean'))


    #if df_dropped.isnull().sum().sum() != 0:
        #print(f"Columns still has some null values {df_dropped.isnull().sum()}")
        
    # Hot en-code male and female
    df_dropped['Sex'] = df_dropped['Sex'].map({'female': 1, 'male': 0}).astype(int)
    
    # hot-encode cabins
    cabin_one_hot = pd.get_dummies(df_dropped['Cabin'], prefix='Cabin')
    df_dropped = df_dropped.drop('Cabin', axis=1)
    df_dropped = df_dropped.join(cabin_one_hot)
    
    # Hot-encode names
    name_one_hot = pd.get_dummies(df_dropped['Title'], prefix='Title')
    df_dropped = df_dropped.drop('Title', axis=1)
    df_dropped = df_dropped.drop('Name', axis=1)
    df_dropped = df_dropped.join(name_one_hot)
    

    # Hot en-code Embarked
    enbarked_one_hot = pd.get_dummies(df_dropped['Embarked'], prefix='Embarked')
    df_dropped = df_dropped.drop('Embarked', axis=1)
    df_dropped = df_dropped.join(enbarked_one_hot)
    
    
    x = np.asarray(df_dropped.drop('Survived', axis=1))
    scale = StandardScaler()
    x_values = scale.fit_transform(x)

    y_label = df_dropped["Survived"].values
    
    return x_values, y_label, df_dropped.drop('Survived', axis=1)

<h2> Preparing Test Set similiar to Training Set</h2>

In [3]:
def data_preprocessing_test(dataframe):
    # Drop the following columns
    df_dropped = dataframe.drop(['PassengerId', 'Ticket'], axis=1)
        
    # Fix Names
    df_dropped['Title'] = df_dropped.Name.str.extract(' ([A-Za-z]+)\.', expand = False)
    Title_Dictionary = {
            "Capt":       "Officer",
            "Col":        "Officer",
            "Major":      "Officer",
            "Dr":         "Officer",
            "Rev":        "Officer",
            "Jonkheer":   "Royalty",
            "Don":        "Royalty",
            "Sir" :       "Royalty",
            "the Countess":"Royalty",
            "Dona":       "Royalty",
            "Lady" :      "Royalty",
            "Mme":        "Mrs",
            "Ms":         "Mrs",
            "Mrs" :       "Mrs",
            "Mlle":       "Miss",
            "Miss" :      "Miss",
            "Mr" :        "Mr",
            "Master" :    "Master"
                       }
    
    df_dropped['Title'] = df_dropped.Title.map(Title_Dictionary)


    # Fix embarked
    df_dropped["Embarked"] = df_dropped["Embarked"].fillna(value="C")
    
    # Fix cabin
    df_dropped['Cabin'] = df_dropped['Cabin'].fillna('U')
    df_dropped['Cabin'] = df_dropped['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    df_dropped['Cabin_T'] = 0


    column = ['Pclass','Sex','Embarked','SibSp','Parch']
    
    for value in range(len(column), 0, -1):
        df_dropped['Age'] = df_dropped['Age'].fillna(df_dropped.groupby(column[0:value])['Age'].transform('mean'))
        df_dropped['Fare'] = df_dropped['Fare'].fillna(df_dropped.groupby(column[0:value])['Fare'].transform('mean'))


    #if df_dropped.isnull().sum().sum() != 0:
        #print(f"Columns still has some null values {df_dropped.isnull().sum()}")
        
    # Hot en-code male and female
    df_dropped['Sex'] = df_dropped['Sex'].map({'female': 1, 'male': 0}).astype(int)
    
    # hot-encode cabins
    cabin_one_hot = pd.get_dummies(df_dropped['Cabin'], prefix='Cabin')
    df_dropped = df_dropped.drop('Cabin', axis=1)
    df_dropped = df_dropped.join(cabin_one_hot)
    
    # Hot-encode names
    name_one_hot = pd.get_dummies(df_dropped['Title'], prefix='Title')
    df_dropped = df_dropped.drop('Title', axis=1)
    df_dropped = df_dropped.drop('Name', axis=1)
    df_dropped = df_dropped.join(name_one_hot)
    

    # Hot en-code Embarked
    enbarked_one_hot = pd.get_dummies(df_dropped['Embarked'], prefix='Embarked')
    df_dropped = df_dropped.drop('Embarked', axis=1)
    df_dropped = df_dropped.join(enbarked_one_hot)
    
    x = np.asarray(df_dropped)
    scale = StandardScaler()
    x_values = scale.fit_transform(x)
    
    return x_values, df_dropped

<h2> Keras method </h2>

In [4]:
df_train_1 = pd.read_csv("train.csv")

x_train, y_label, dataframe=data_preprocessing_train(df_train_1)

In [5]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

model = Sequential([
    Dense(x_train.shape[1], activation="relu",input_dim=x_train.shape[1]),
    Dropout(0.20),
    Dense(10, activation="relu"),
    Dropout(0.20),
    Dense(8, activation="relu"),
    Dropout(0.20),
    Dense(6, activation="relu"),
    Dropout(0.20),
    Dense(3, activation="relu"),
    Dropout(0.20),
    Dense(1, activation="sigmoid")

])

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])


model.fit(x_train, 
          y_label, 
          epochs=100,
          callbacks=[tensorboard_callback])

print(f"Model evaluation value is {model.evaluate(x_train, y_label)[1]}")

# Model evaluation value is 0.868686854839325

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 891 samples
Epoch 1/100
891/891 [==============================] - 0s 500us/sample - loss: 0.6948 - acc: 0.5802
Epoch 2/100
891/891 [==============================] - 0s 84us/sample - loss: 0.6882 - acc: 0.6139
Epoch 3/100
891/891 [==============================] - 0s 81us/sample - loss: 0.6767 - acc: 0.6184
Epoch 4/100
891/891 [==============================] - 0s 84us/sample - loss: 0.6673 - acc: 0.6229
Epoch 5/100
891/891 [==============================] - 0s 83us/sample - loss: 0.6559 - acc: 0.6240
Epoch 6/100
891/891 [==============================] - 0s 85us/sample - loss: 0.6326 - acc: 0.6352
Epoch 7/100
891/891 [==============================] - 0s 79us/sample - loss: 0.6202 - acc: 0.6577
Epoch 8/100
891/891 [==============================] - 0s 81us/sample - loss: 0.6052 - acc: 0.6869
Epoch 9/100

In [6]:
df_test = pd.read_csv("test_survi.csv")
y_test= df_test["Survived"].values

df_test_cleaned = df_test.drop('Survived', axis=1)
x_test, dataframe_test=data_preprocessing_test(df_test_cleaned)

results = model.evaluate(x_test, y_test)
print('Test Acc on actual results:', results[1])

418/418 [==============================] - 0s 45us/sample - loss: 0.5135 - acc: 0.7488
Test Acc on actual results: 0.74880385


In [7]:
# If happy, execute this block and submit to kaggle
df_test = pd.read_csv("test.csv")
x_test=data_preprocessing_test(df_test)

#fit with all values for submission:
classes = model.predict_classes(x_test)
df=pd.DataFrame(classes)

submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": df[0]
    })
submission.to_csv('titanic_keras.csv', index=False)

<h2> Another Methods - Random Forest</h2>

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

df_train = pd.read_csv("train.csv")

x_train, y_train, dataframe=data_preprocessing_train(df_train)

# # Set our parameter grid
# param_grid = {
#     'max_features': [10,11,12,13,14,15,16,17,18,19,20,21,22,23],
#     'max_depth': [3,5,7,9,11,13,15,16,17,18,19,20,21,22,23]

# }
# randomForest = RandomForestClassifier(random_state = 2)

# randomForest_CV = GridSearchCV(estimator = randomForest, param_grid = param_grid, cv = 5)
# randomForest_CV.fit(x_train, y_train)

# randomForest_CV.best_params_

#random_forest.score(x_train, y_train)

# acc_random_forest = round(random_forest.score(x_train, y_train) * 100, 2)
# print(round(acc_random_forest,2,), "%")

{'max_depth': 7, 'max_features': 17}

In [40]:
randomForest = RandomForestClassifier(random_state = 2)

randomForest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=2, verbose=0,
                       warm_start=False)

In [41]:
# Check with real data
from sklearn.metrics import accuracy_score 

df_test_survi = pd.read_csv("test_survi.csv")
df_test_cleaned = df_test_survi.drop('Survived', axis=1)
true_label= df_test_survi["Survived"].values

X_test, dataframe =data_preprocessing_test(df_test_cleaned)

y_pred = randomForest.predict(X_test)

accuracy_score(true_label, y_pred)

0.7703349282296651

In [ ]:
# If happy, submit this to kaggle
df_test = pd.read_csv("test.csv")
X_test=data_preprocessing_test(df_test)


y_pred = random_forest.predict(X_test)
df=pd.DataFrame(y_pred)

submission = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": df[0]
    })

submission.to_csv('titanic_randomF.csv', index=False)